In [1]:
import os
import sys

spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()

import pyspark

In [2]:
number_cores = 8
memory_gb = 8
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [3]:
logs = sc.textFile("./data/nsf/*").cache()

In [144]:
#echo %cd%

In [143]:
#logs.take(10)

In [142]:
import glob

data_path = "./data/nsf"
all_files = glob.glob("./data/nsf/*.xml")

file_names = sc.parallelize(all_files)
#file_names.collect()

In [141]:
#file_names.take(5)

In [15]:
s = './data/nsf\\1200004.xml'
s = s.split("\\")[1]

In [140]:
#s

In [139]:
raw_data = file_names.map(lambda f: (f.split("\\")[1], open(f,mode='r').read()))
#raw_data.take(5)

In [121]:
s = raw_data.take(1)

In [123]:
import xml.etree.ElementTree as ET

In [124]:
elem = ET.fromstring(s[0][1])

In [145]:
#for c1 in elem:
#    for c2 in c1:
#        print(c2.tag)
#        for c3 in c2:
#            print("\t", c3.tag)

In [155]:
import xml.etree.ElementTree as ET
def extractUsefulData(y):
    out = []
    try:
        elem = ET.fromstring(y[1])
        for c1 in elem:
            for c2 in c1:
                if (c2.tag == "AwardTitle"):
                    out.append((c2.tag, c2.text))
                if (c2.tag == "AwardEffectiveDate"):
                    out.append((c2.tag, c2.text))
                if (c2.tag == "AwardAmount"):
                    out.append((c2.tag, c2.text))
                for c3 in c2:
                    if (c2.tag == "Investigator" and c3.tag == "NSF_ID"):
                        out.append((c2.tag, c3.tag, c3.text))
                    if (c2.tag == "Performance_Institution" and c3.tag == "Name"):
                        out.append((c2.tag, c3.tag, c3.text))
                    if (c2.tag == "ProgramElement" and c3.tag == "Text"):
                        out.append((c2.tag, c3.tag, c3.text))
                    if (c2.tag == "Appropriation" and c3.tag == "Name"):
                        out.append((c2.tag, c3.tag, c3.text))
    except:
        out.append(("Exception"))
    return out

In [156]:
useful_data = raw_data.map(lambda x: extractUsefulData(x))

In [157]:
useful_data.take(1)

[[('AwardTitle',
   'Collaborative Research: Investigation of Odor-triggered Neuronal Dynamics and Experience-induced Olfactory Learning'),
  ('AwardEffectiveDate', '09/15/2012'),
  ('AwardAmount', '355586'),
  ('Investigator', 'NSF_ID', '000601625'),
  ('Performance_Institution', 'Name', 'University of Arizona'),
  ('ProgramElement', 'Text', 'MATHEMATICAL BIOLOGY'),
  ('ProgramElement', 'Text', 'NIGMS'),
  ('Appropriation', 'Name', 'NSF RESEARCH & RELATED ACTIVIT'),
  ('Appropriation', 'Name', 'NSF RESEARCH & RELATED ACTIVIT'),
  ('Appropriation', 'Name', 'NSF RESEARCH & RELATED ACTIVIT')]]

### Who has won multiple awards

In [23]:
for c1 in elem:
    for c2 in c1:
        #print(c2.tag, c2.text)
        for c3 in c2:
            if (c3.tag == 'NSF_ID'):
                print("\t", c3.tag, c3.text)

	 NSF_ID 000601625


In [ ]:
for n in elem.findall("./Award/Investigator/NSF_ID"):
    print(n.text)

In [75]:
import xml.etree.ElementTree as ET

def getIDCount(e):
    PIs = []
    try:
        elem = ET.fromstring(e[1])
        for n in elem.findall("./Award/Investigator/NSF_ID"):
            if (n.text != ''):
                PIs.append((n.text,1))
    except:
        PIs.append(('Exception',1))
    return PIs
getIDCount(s[0])

[('000601625', 1)]

In [ ]:
tmp = raw_data.flatMap(getIDCount)
tmp.count()

In [ ]:
tmp.take(5)

In [ ]:
getAllCount = tmp.countByKey()
getAllCount.g

In [ ]:
getAllCount['Exception']

In [ ]:
import xml.etree.ElementTree as ET

def getIDCount2(e):
    PIs = []
    try:
        elem = ET.fromstring(e[1])
        for n in elem.findall("./Award/Investigator/NSF_ID"):
            if (n.text != ''):
                PIs.append((n.text,1))
    except:
        PIs.append((e,0))
    return PIs
getIDCount(s[0])

In [ ]:
bad_data = raw_data.flatMap(getIDCount2).filter(lambda x: x[1] == 0)
bad_data.take(1)

In [ ]:
sb = bad_data.take(1)

In [ ]:
sb[0][0][1]

In [ ]:
lines = sb[0][0][1].split('\n')

In [ ]:
lines = sb[0][0][1].split('\n')
lines

In [ ]:
good_lines = s[0][1].split('\n')
good_lines

In [ ]:
s[0][1]

In [ ]:
len(lines)

In [ ]:
new_line = sb[0][0][1].split('<POR>',1)[0] + "</Award>\n</rootTag>\n"

In [ ]:
elem = ET.fromstring(new_line)
for c1 in elem:
    for c2 in c1:
        print(c2.tag, c2.text)
        for c3 in c2:
            print("\t", c3.tag, c3.text)

In [ ]:
import xml.etree.ElementTree as ET

def getIDCount3(e):
    PIs = []
    try:
        elem = ET.fromstring(e[1])
    except:
        elem = ET.fromstring(e[1].split('<POR>',1)[0] + "</Award>\n</rootTag>\n")

    for n in elem.findall("./Award/Investigator/NSF_ID"):
            if (n.text != ''):
                PIs.append((n.text,1))
    return PIs
print(getIDCount3(s[0]))
print(getIDCount3(sb[0][0]))

In [ ]:
tmp = raw_data.flatMap(getIDCount)
tmp.count()

In [ ]:
NSF = tmp.reduceByKey(lambda x, y: x + y).cache()
NSF.count()

In [ ]:
NSF.takeOrdered(50, key = lambda x: -x[1])